<center><font size = "10"> Week 10 - Cells in Space  <center>
<center><font size = "8">Home work - Solutions<center>

<font size = "3">Please, write your group number and the names and surnames of the group members here
<br>Group number: 8
<br>Group member1: Chiara Maria Matti
<br>Group member2: Elena Grazia Gado
<br>Group member3: Eliser Josan Romero

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import neurom as nm
from neurom import view

sns.set_style('darkgrid')

# Load a morphology
morph = nm.load_morphology('Pyr_01.swc')

# Check that it is not centered
print('soma center:',morph.soma.center)
# Visualize it
plot_morphologies([morph])


## <font color= 'red'>Ex 1:</font> Placing cells

<font size='3'> 10um_cells.csv Is a table with (among other information) the positions of neurons in a model of the primary visual cortex (within the cooridnate system of the allen mouse brain atlas).
   The ascii files contain the morphologies of these cells (the number corresponds to the row in the csv).
    
Load the morphologies and dataframe and place the cells at the x, y, z positions reported in the table.
Visualize the morphologies in all three planes.

<font size='3'><font color= 'red'>[1 pts]</font>

## <font color= 'red'>Ex 2:</font> Rotate cells

Align each cell's y-axis with the vector 'u', 'v', 'w'  reported for it in 21_cells.csv.
At the end of this part their soma positions should still be the same as they were at the end of Ex 1.
Visualize them again in all three planes.

Hint: consider the order of rotation/translation operations.

<span style="color:green">
<font size='3'>

### **Grading**: 

- 1 pts for correct rotation and positon
- 0.5 pts for plotting 3 axes

</span>

## <font color= 'red'>Ex 3:</font> Synaptic pruning rules
<font size=4>
To anatomically constrain the connectivity between the neurons using this geometry we need to identify the potential synapse sites between them. 
However, going from potential synapse sites to actual synapse sites is generally not straightforward.

We are not going to implement the above algorithm, instead we will do something simpler.

Suppose in the brain region we are modeling we have only very coarse-grained data on connection probability: the overall connection probability for cells with soma within 10um of an inserted electrode. This data reveals an overall connection probability of 0.05. Additionally, we have some data that the number of synapses per connection within this brain region tends to be a poisson distribution with a mean of 6.
    
To add the anatomical constraint, we apply a simple version of Peters' rule where we assume that the expectation value of the number of synapses between a pair of neurons is proportional to the number of potential synapse sites.

$E[n_{syn}|n_{app}=\hat{n_{app}}] = C \hat{n_{app}}$
    
Where $n_{syn}$ and $n_{app}$ are the number of synapses and appositions(potential synapse sites) respectively, and C is a constant.
    
Conditional expectation value is defined as:
    
    
$ E[n_{syn}|n_{app}=\hat{n_{app}}] = \sum_{\hat{n_{syn}} = 0}^{\infty} \hat{n_{syn}} P(n_{syn}=\hat{n_{syn}}|n_{app}=\hat{n_{app}})$
    
The simplest solution that satisfies this constraint is to have a fixed probability of converting a potential synapse site into an actual synapse, leading to a binomial distribution.

$P(n_{syn}=\hat{n_{syn}}|n_{app}=\hat{n_{app}}) = Binom(n=\hat{n_{app}}, k=\hat{n_{syn}}, p=C)$
    
We recieve some data that the average density of synapses is 28% of the density of potential synapse sites, setting C at 0.28.

appositions.csv contains all potential synapse sites identified between the 21 neurons we placed before.
Apply this general pruning and provide a dataframe of actualy synapses based on it.

Subsequently:
    
    - calculate the overall connection probability. 
    - plot a histogram of the number of synapses per connection. Ensure you use a bin size of 1 (so 1, 2, 3, 4 etc.)
    - on the same axis, plot the probability mass function of a poisson dstribution at 6 synapses per connection.

Based on these outcomes, what are some limitations of this naive synapse placement rule? Can you suggest one or more improvements that would lead to more realistic connectivity?

<font color='red'> [3 pts]
    
Further reading: 
    This problem gets far more complex when you consider that each brain region consists of many cell types with differing connectivity properties.
    for an example of how this kind of problem can be addressed in practice, see this paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4597796/ 

<span style="color:green">
<font size='3'>

### **Grading**: 

- 0.5 pt for implementing the rule.
- 0.5 pts each for connection probability and synapses per connection.
- 1 pts for answering the questions

</span>

In [1]:
appositions = pd.read_csv("toy_data/10um_appositions.csv")
appositions.head()


,population,edge_ids,@source_node,@target_node,efferent_section_id,efferent_surface_z,afferent_center_y,efferent_center_z,efferent_segment_id,afferent_surface_z,...,afferent_section_type,afferent_section_id,afferent_section_pos,afferent_surface_y,afferent_segment_offset,efferent_surface_y,afferent_surface_x,efferent_center_y,efferent_section_pos,efferent_center_x
0,V1_neurons__V1_neurons__chemical_synapse,2997519,19473,2636,151,8481.373,804.57070,8481.260,48,8482.396,...,3,613,0.781682,804.47630,0.159011,803.49506,9179.472,803.36770,0.773995,9180.0625
1,V1_neurons__V1_neurons__chemical_synapse,2997911,45243,2636,84,8496.062,850.48200,8495.991,7,8496.603,...,3,619,0.771799,850.46454,0.946993,850.26740,9184.358,850.25750,0.705606,9186.2320
2,V1_neurons__V1_neurons__chemical_synapse,2997912,45243,2636,176,8439.580,896.38336,8439.572,15,8438.884,...,3,588,0.998704,896.40380,0.846684,896.60754,9161.349,896.65440,0.974271,9160.3470
3,V1_neurons__V1_neurons__chemical_synapse,2997913,45243,2636,177,8440.750,895.43353,8440.821,3,8439.062,...,3,588,0.976641,895.47076,0.960646,895.92926,9161.347,895.98376,0.018393,9160.9710
4,V1_neurons__V1_neurons__chemical_synapse,2997914,45243,2636,178,8440.553,895.00507,8440.653,5,8439.190,...,3,588,0.966387,895.04535,0.512062,895.39294,9161.430,895.45030,0.040814,9161.7720


In [2]:
appositions.describe()

,edge_ids,@source_node,@target_node,efferent_section_id,efferent_surface_z,afferent_center_y,efferent_center_z,efferent_segment_id,afferent_surface_z,afferent_segment_id,...,afferent_section_type,afferent_section_id,afferent_section_pos,afferent_surface_y,afferent_segment_offset,efferent_surface_y,afferent_surface_x,efferent_center_y,efferent_section_pos,efferent_center_x
count,2.356000e+03,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,...,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000
mean,3.446069e+08,69187.112054,80288.396435,221.123939,8364.502414,1036.600833,8364.501282,22.098472,8364.476691,32.506367,...,3.438879,290.353141,0.469767,1036.636299,0.532941,1036.615342,9110.092333,1036.616843,0.423501,9110.104817
std,2.984318e+08,48726.036914,50244.631908,219.081408,108.972847,198.370587,108.977079,37.299597,108.940684,38.175441,...,0.711506,188.000095,0.280296,198.372180,0.346525,198.351931,58.667749,198.352392,0.299319,58.674102
min,2.997519e+06,2636.000000,2636.000000,1.000000,8033.813000,788.159900,8033.699000,0.000000,8033.848000,0.000000,...,1.000000,0.000000,0.000000,788.062900,0.000000,787.912540,8894.895500,787.860600,0.000000,8894.539000
25%,1.177201e+08,45243.000000,45243.000000,56.000000,8309.450500,892.495225,8309.393750,2.000000,8309.652750,10.000000,...,3.000000,161.000000,0.239071,892.432275,0.246426,892.174675,9077.725000,892.214430,0.154158,9076.571000
50%,1.975945e+08,49867.000000,62482.000000,151.000000,8415.813500,938.612820,8415.801000,9.000000,8416.346000,23.000000,...,4.000000,283.000000,0.462964,937.740000,0.525720,937.797315,9129.940000,937.803770,0.390076,9129.958500
75%,5.828839e+08,111927.000000,132720.000000,294.000000,8441.409500,1151.230475,8441.383250,25.000000,8441.301500,43.000000,...,4.000000,374.000000,0.692090,1151.290950,0.812228,1151.188125,9151.172500,1151.171525,0.667515,9151.241000
max,9.098867e+08,171009.000000,171009.000000,934.000000,8525.069000,1627.118300,8525.106000,443.000000,8524.313000,359.000000,...,4.000000,1017.000000,0.999801,1627.187100,1.419463,1627.555200,9218.353000,1627.600000,0.999532,9217.388000


@source_node  @target_node
2636          16297            9
              19473            9
              45243           12
              49867            1
              54379            3
                              ..
169253        157828           1
171009        149948           4
              157828           6
              168462           3
              169253           3
Name: edge_ids, Length: 139, dtype: int64

<font color='blue'>
    Answers:
        This kind of synapse placement rule will overestimate synapses per connection and wind up placing a lot of connections with only one synapse, which should be in the minority in reality.
        Any suggestion which would shift the mode of the distribution of synapses per connecton higher and reduce connection probability is accepted.